# Steps to follow

1.train a generative adversarial network on a one-dimensional function                             
2.define the standalone discriminator model                                   
3.define the standalone generator model                                            
4.define the combined generator and discriminator model, for updating the generator                    
5.generate n real samples with class labels                                               
6.generate points in latent space as input for the generator                                      
7.use the generator to generate n fake examples, with class labels                                        
8.evaluate the discriminator and plot real and fake points                                               
9.train the generator and discriminator                   


In [3]:
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam

from matplotlib import pyplot
import numpy as np
from numpy.random import randn

Using TensorFlow backend.


In [4]:
# define the standalone discriminator model

def discriminator(n_inputs = 2):
    model = Sequential()
    model.add(Dense(25, activation = 'relu', kernel_initializer='he_uniform',input_dim = n_inputs))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'] )
    return model

In [5]:
# define the standalone generator model

def generator(latent_dim,n_outputs = 2):
    
    model = Sequential()
    model.add(Dense(15,activation = 'relu',kernel_initializer='he_uniform',input_dim = latent_dim))
    model.add(Dense(n_outputs, activation = 'linear'))
    return model
    

In [6]:
# define the combined generator and discriminator model, for updating the generator

def gan(generator, discriminator):
    
    discriminator.trainable = False
    
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model



In [7]:
# generate n real samples with class labels
# sample
def generate_real(n):
    x1 = np.random.rand(n) - 0.5
    x2 = x1 * x1
    x1 = x1.reshape(n,1)
    x2 = x2.reshape(n,1)
    x = np.hstack((x1,x2))
    y = np.ones((n,1))
    return x,y

In [8]:
# generate points in latent space as input for the generator
def generate_points(latent_dim , n):
    
    inputs = randn(latent_dim * n)
    inputs = inputs.reshape(n , latent_dim)
    return inputs

In [9]:
# use the generator to generate n fake examples, with class labels

def generate_fake(generator, latent_dim, n):
    
    inputs = generate_points(latent_dim, n)
    
    x = generator.predict(inputs)
    y = np.zeros((n,1))
    
    return x,y

In [10]:
# evaluate the discriminator and plot real and fake points
def evaluate(epoch, generator, discriminator, latent_dim, n=100):
    
    x_real,y_real = generate_real(n)
    _ , acc_real = discriminator.evaluate(x_real,y_real, verbose=0)
    
    x_fake,y_fake = generate_fake(generator,latent_dim ,n)
    _ , acc_fake = discriminator.evaluate(x_fake,y_fake , verbose=0)
    
    print (epoch, acc_real,acc_fake)
    
    pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
    pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()
    

In [11]:
# train the generator and discriminator
def train(g_model,d_model,gan_model,latent_dim, n_epochs=10000, n_batch=128,n_eval=2000):
    
    # determine half the size of one batch, for updating the discriminator
    half_batch = int(n_batch / 2)
# manually enumerate epochs
    for i in range(n_epochs):
        
# prepare real samples
        x_real, y_real = generate_real(half_batch)
# prepare fake examples
        x_fake, y_fake = generate_fake(g_model, latent_dim, half_batch)
# update discriminator
        d_model.train_on_batch(x_real, y_real)
        d_model.train_on_batch(x_fake, y_fake)
# prepare points in latent space as input for the generator
        x_gan = generate_points(latent_dim,n = n_batch)
# create inverted labels for the fake samples
        y_gan = np.ones((n_batch, 1))
# update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan, y_gan)
# evaluate the model every n_eval epochs
        if (i+1) % n_eval == 0:
            evaluate(i, g_model, d_model, latent_dim)

In [12]:
latent_dim = 5
# create the discriminator
discriminator = discriminator()
# create the generator
generator = generator(latent_dim)
# create the gan
gan_model = gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)


W0601 00:37:48.296775 13164 deprecation.py:323] From C:\Users\samya\Anaconda3\lib\site-packages\tensorflow_core\python\ops\nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
C:\Users\samya\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1999 0.5299999713897705 0.4399999976158142
3999 0.49000000953674316 0.029999999329447746
5999 0.5 0.8700000047683716
7999 0.3499999940395355 0.7699999809265137
9999 0.44999998807907104 0.5799999833106995
